<a href="https://colab.research.google.com/github/bstrain71/422_MNIST/blob/master/Strain_Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
test_df = pd.read_csv('/content/test.csv')
train_df = pd.read_csv('/content/train.csv')

In [20]:
print(train_df.shape) # 42000x785? OK
print(test_df.shape) # 28000x784? OK

(42000, 785)
(28000, 784)


In [23]:
X_train = train_df.drop(columns=['label'])
y_train = train_df.drop(columns!=['label'])
y_train.head()

NameError: ignored

In [0]:
start=datetime.now()
rf2.fit(trainimages,labels)
end=datetime.now()
print(end-start)